# Demonstration of using EasyEdit (requires GPU)

In [ ]:
## Clone Repo and install
!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!pip install datasets==1.18.3 einops==0.4.0 gpustat==1.1 hydra-core==1.1.1 higher==0.2.1 importlib-metadata==6.3.0 matplotlib==3.5.1 nltk==3.6.5 numpy==1.22.1 omegaconf==2.1.1 pandas==1.4.0 PyYAML==6.0 scikit-learn==1.0.2 sentence-transformers==2.2.2 tokenizers>=0.12.1 torch==1.11.0 tqdm==4.62.3 transformers==4.30.1 openai==0.27.9

In [ ]:
from easyeditor import BaseEditor
from easyeditor import ROMEHyperParams


# make an edit using GPT2-XL
prompts = [
    'Suzuki APV is developed by',
]
ground_truth = [
    'Suzuki APV',
]
target_new = [
    'Nokia',
]
subject = [
    'Suzuki APV',
]

hparams = ROMEHyperParams.from_hparams('./hparams/ROME/gpt2-xl')
hparams.model_name='gpt2-xl'
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)

In [ ]:
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
generation_prompts = [
    "Suzuki APV is developed by"
]
batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=300,
    do_sample=True,
    repetition_penalty=1.5,
    top_k=5,
    top_p=0.95
)
print('Post-Edit Outputs: ', tokenizer.decode(post_edit_outputs.detach().cpu().numpy().tolist()[0], skip_special_tokens=True))